# Amazon Nova

In [2]:
import json

import boto3

session = boto3.Session(profile_name="gen-ai-ops", region_name="us-east-1")
bedrock_client = session.client(service_name="bedrock")
bedrock_runtime_client = session.client(service_name="bedrock-runtime")

In [1]:
PRO_MODEL_ID = "amazon.nova-pro-v1:0"  # TEXT & VISION
LITE_MODEL_ID = "amazon.nova-lite-v1:0"  # TEXT & VISION
MICRO_MODEL_ID = "amazon.nova-micro-v1:0"  # TEXT

#### Synchronous API Call

In [10]:
system_list = [{"text": "You should respond all the messages in French."}]
message_llist = [
    {
        "role": "user",
        "content": [
            {
                "text": "Youtube is a great platform to learn coding and technology in the field of computer science."
            }
        ],
    }
]

config = {"max_new_tokens": 300, "top_p": 0.9, "top_k": 20}
request = {"messages": message_llist, "system": system_list, "inferenceConfig": config}

response = bedrock_runtime_client.invoke_model(
    modelId=LITE_MODEL_ID, body=json.dumps(request)
)
print(response)

request_id = response["ResponseMetadata"]["RequestId"]
print(request_id)

body = json.loads(response["body"].read())
print(body)

content_text = body["output"]["message"]["content"][0]["text"]
print("\n", content_text)

{'ResponseMetadata': {'RequestId': '5e4a0385-511e-4f57-bc7a-0e8ecd761b72', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 01 Oct 2025 14:34:57 GMT', 'content-type': 'application/json', 'content-length': '915', 'connection': 'keep-alive', 'x-amzn-requestid': '5e4a0385-511e-4f57-bc7a-0e8ecd761b72', 'x-amzn-bedrock-invocation-latency': '1104', 'x-amzn-bedrock-cache-write-input-token-count': '0', 'x-amzn-bedrock-cache-read-input-token-count': '0', 'x-amzn-bedrock-output-token-count': '147', 'x-amzn-bedrock-input-token-count': '26'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'body': <botocore.response.StreamingBody object at 0x000001BF343B5C00>}
5e4a0385-511e-4f57-bc7a-0e8ecd761b72
{'output': {'message': {'content': [{'text': "YouTube est une excellente plateforme pour apprendre le codage et la technologie dans le domaine de l'informatique. De nombreux créateurs de contenu proposent des tutoriels détaillés, des cours complets et des explications sur divers sujets tels qu

#### Streaming API Call

In [14]:
from datetime import datetime as dt

system_list = [
    {
        "text": "Act as a writing assistant. When the user provides you with a topic, write a short story about that topic"
    }
]

message_llist = [{"role": "user", "content": [{"text": "A trip to Ladakh."}]}]

config = {"max_new_tokens": 1000, "top_p": 0.9, "top_k": 20}

request = {"messages": message_llist, "system": system_list, "inferenceConfig": config}

start_time = dt.now()

response = bedrock_runtime_client.invoke_model_with_response_stream(
    modelId=LITE_MODEL_ID, body=json.dumps(request)
)

chunk_count = 0
time_to_first_token = None

stream = response.get("body")
if stream:
    for event in stream:
        chunk = event.get("chunk")
        if chunk:
            chunk_json = json.loads(chunk.get("bytes").decode())
            content_block_delta = chunk_json.get("contentBlockDelta")
            if content_block_delta:
                if time_to_first_token is None:
                    time_to_first_token = dt.now() - start_time
                    print(f"Time to first token: {time_to_first_token}")
                chunk_count += 1
                current_time = dt.now().strftime("%d-%m-%Y %H:%M:%S:%f")
                print(content_block_delta.get("delta").get("text"), end="")
    print("\nTotal token count: ", chunk_count)
else:
    print("No response stream received.")

Time to first token: 0:00:00.402748
The sun had just begun to rise over the jagged peaks of the Himalayas, casting a golden hue on the rugged landscape of Ladakh. Maya stood at the edge of her hotel balcony, her breath visible in the crisp morning air. She had always dreamed of visiting this remote region, known for its stark beauty and spiritual aura.

As she stepped out onto the narrow, winding roads, the first thing that struck her was the silence. The air was so thin and pure that even the sound of her footsteps seemed muffled. The landscape was a patchwork of arid mountains, shimmering lakes, and verdant valleys. Each turn revealed a new, breathtaking vista.

Maya's journey began with a visit to the Magnetic Hill, a peculiar phenomenon where gravity seemed to defy logic, and vehicles rolled uphill. She laughed as she watched a local shepherd struggle to keep his yak from rolling back down the slope. 

Next, she ventured to the Shanti Stupa, a white dome-shaped monument that offere

#### Image Understanding